# **Introdução aos Modelos DSGE**

João Ricardo Costa Filho \\
https://www.joaocostafilho.com

# **Modelo Novo Keynesiano**

## Instale na máquina "virtual" o Octave e o Dynare.

In [ ]:
!sudo apt-get update  &> /dev/null
!sudo apt-get install -y x13as  &> /dev/null

In [ ]:
!apt-get install octave &> /dev/null

In [ ]:
!apt-get install dynare &> /dev/null

## Equilíbrio

In [ ]:
%%writefile NK_SS.mod

%--------------------------------------------------------------------------------------------------------------------------------------
% 1. Definição das Variáveis
%--------------------------------------------------------------------------------------------------------------------------------------


var pi           ${\pi}$     (long_name='taxa de inflação')
    x            ${\hat{x}}$ (long_name='hiato do produto')
    yn           ${y^{n}}$   (long_name='produto natural')
    y            ${y}$       (long_name='PIB')
    rn           ${r^{n}}$   (long_name='taxa de juros natural')
    i            ${i}$       (long_name='taxa de juros nominal')
    h            ${h}$       (long_name='horas trabalhadas')
    nu           ${\nu}$     (long_name='choque monetário')
    A            ${A}$       (long_name='produtividade')
;


varexo eA ${\varepsilon^A}$   (long_name='choque de produtividade')
       em ${\varepsilon^m}$   (long_name='choque de política monetária')
;

%--------------------------------------------------------------------------------------------------------------------------------------
% 2. Calibração
%--------------------------------------------------------------------------------------------------------------------------------------

parameters alpha   ${\alppha}$    (long_name='parâmetro da função de produção')
           beta    ${\beta}$      (long_name='fator de desconto')
           rho_A   ${\rho_a}$     (long_name='autocorrelação da produtividade')
           rho_nu  ${\rho_{\nu}}$ (long_name='autocorrelação dp choque de política monetária')
           sigma   ${\sigma}$     (long_name='curvatura da função utilidade')
           phi     ${\phi}$       (long_name='unitary Frisch elasticity')
           phi_pi  ${\phi_{\pi}}$ (long_name='inflation feedback Taylor Rule')
           phi_y   ${\phi_{y}}$   (long_name='output feedback Taylor Rule')
           eta     ${\eta}$       (long_name='semi-elasticity of money demand')
           epsilon ${\epsilon}$   (long_name='demand elasticity')
           theta   ${\theta}$     (long_name='Parâmetro da rigidez nominal de preços (Calvo)')
;

sigma   = 2;
phi     = 1;
phi_pi  = 1.5;
phi_y   = .5/4;
theta   = 2/3;
rho_nu  = 0.5;
rho_A   = 0.9;
beta    = 0.97;
eta     = 4;
alpha   = 0.44;
epsilon = 6;

%--------------------------------------------------------------------------------------------------------------------------------------
% 3. Modelo
%--------------------------------------------------------------------------------------------------------------------------------------

model(linear);

# Omega    = ( 1 - alpha ) / ( 1 -alpha + alpha * epsilon );
# psi_n_ya = ( 1 + phi ) / ( sigma * ( 1 - alpha ) + phi + alpha);
# lambda   = ( 1 - theta ) * ( 1 - beta * theta ) / theta * Omega;
# kappa    = lambda * ( sigma + ( phi + alpha ) / ( 1 - alpha ) );

[name = 'Curva de Phillips Novo Keynesiana / Curva de Oferta Agregada']
pi = beta * pi(+1) + kappa * x;

[name = 'Curva IS Novo Keynesiana']
x = - 1 / sigma * ( i - pi(+1) - rn ) + x(+1);

[name = 'Regra de Política Monetária']
i = phi_pi * pi + phi_y * x + nu;

[name = 'Taxa de Juros Natural']
rn = sigma * psi_n_ya* ( A(+1) - A );

[name = 'Produto Natural']
yn = psi_n_ya * A;

[name = 'Hiato do produto']
x = y - yn;

[name = 'Choque de política monetária']
nu = rho_nu * nu(-1) + em;

[name = 'Produtividade']
A = rho_A * A(-1) + eA;

[name = 'Função de Produção']
y = A + ( 1 - alpha ) * h;

end;

%--------------------------------------------------------------------------------------------------------------------------------------
% 4. Equilíbrio
%--------------------------------------------------------------------------------------------------------------------------------------

steady;
check;
model_diagnostics;
model_info;

Overwriting NK_SS.mod


In [ ]:
%%writefile solve.m
dynare NK_SS.mod

Overwriting solve.m


In [ ]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 9 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

pi 		 0
x  		 0
yn 		 0
y  		 0
rn 		 0
i  		 0
h  		 0
nu 		 0
A  		 0

EIGENVALUES:
         Modulus             Real        Imaginary

             0.5              0.5                0
             0.9              0.9                0
           1.098            1.083           0.1773
           1.098            1.083          -0.1773
       6.944e+16       -6.944e+16                0


There are 3 eigenvalue(s) larger than 1 in modulus 
for 3 forward-looking variable(s)

The rank condition is verified.

MODEL_DIAGNOSTICS:  No obvious problems with this mod-file were detecte

## Funções Impulso-resposta

In [ ]:
%%writefile NK.mod

@#include "NK_SS.mod"

%--------------------------------------------------------------------------------------------------------------------------------------
% 5. Simulação
%--------------------------------------------------------------------------------------------------------------------------------------

shocks;
   var em = 0.25^2;
   var eA = 0.01^2;
end;

stoch_simul(ar=1, order=1, irf=20, graph_format = pdf);

Overwriting NK.mod


In [ ]:
%%writefile solve.m
dynare NK.mod

Overwriting solve.m


In [ ]:
!octave solve.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
Starting Dynare (version 5.0).
Calling Dynare with arguments: none
Starting preprocessing of the model file ...
Found 9 equation(s).
Evaluating expressions...done
Computing static model derivatives (order 1).
Computing dynamic model derivatives (order 2).
Processing outputs ...
done
Preprocessing completed.


STEADY-STATE RESULTS:

pi 		 0
x  		 0
yn 		 0
y  		 0
rn 		 0
i  		 0
h  		 0
nu 		 0
A  		 0

EIGENVALUES:
         Modulus             Real        Imaginary

             0.5              0.5                0
             0.9              0.9                0
           1.098            1.083           0.1773
           1.098            1.083          -0.1773
       6.944e+16       -6.944e+16                0


There are 3 eigenvalue(s) larger than 1 in modulus 
for 3 forward-looking variable(s)

The rank condition is verified.

MODEL_DIAGNOSTICS:  No obvious problems with this mod-file were detecte